## Dashboard Creation and Automated Reporting Instruction Manual

### Required Application Downloads

### Grab and Open Files

The cells inbetween the lines gets the data merges it moves the employees with multiple entries to the bottom of the table and then consolidates that specific multiple entry data and calculated the numbers off the merged data.

***

In [335]:
#Library Importation
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np

#Loading Necessary Workbooks
wb1 = load_workbook ("StaffPerformanceOverviewCR06-23-23 test copy.xlsx")
wb2 = load_workbook("StaffPerformanceOverviewNH06-23-23 test copy.xlsx")
ws1 = wb1.active
ws2 = wb2.active

#Appending Data To One Workbook
for row in ws2.iter_rows(min_row=2, values_only=True): 
    ws1.append(row)

wb1.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

#reload new workbook that was just saved
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")
ws = wb.active

#Remove Row to consolidate data
start_row = 12
num_rows_to_remove = 1
ws.delete_rows(start_row, num_rows_to_remove)

# Define the column letter you want to update (e.g., column A)
column_letter = 'E'

# Define the value to be replaced and its corresponding replacement
old = "NA"
new = "0"

# Get the column index from the column letter
column_index = ord(column_letter) - ord('A') + 1

# Loop through the cells in the selected column and replace the old value with the new value
for row in ws.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
    cell = row[0]
    if cell.value == old:
        cell.value = new

from openpyxl.styles import NamedStyle

# Create named style for the whole number format
whole_number_style = NamedStyle(name="whole_number", number_format="0")

# Columns that need to be formatted as a whole number (example: columns A, B, and D)
columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

# Apply the named style to each cell in the selected columns
for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = whole_number_style

Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

for column_letter in columns_to_format:
    for cell in ws[column_letter]:
        cell.style = Decimal_Number_Style

wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

In [336]:
from openpyxl import load_workbook

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the ws you want to update (assuming it's the first sheet)
ws = wb.active

# Find the last row in the ws
last_row = ws.max_row

# Create a dictionary to store the rows of employees with multiple entries
multiple_entries_rows = {}

# Iterate through the rows and collect the rows of employees with multiple entries
for row in range(3, last_row + 1):
    employee_id = ws.cell(row=row, column=1).value  # Assuming 'EmployeeID' is in column A
        if employee_id in multiple_entries_rows:
            multiple_entries_rows[employee_id].append([cell.value for cell in ws[row]])
        else:
            multiple_entries_rows[employee_id] = [[cell.value for cell in ws[row]]]

# Clear the existing data in the ws
ws.delete_rows(3, last_row)

# Write the rows of employees with single entries at the top of the ws
starting_row_for_multiple_entries = 11  # Adjust this variable for different reports
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) == 1:
        ws.append(rows[0])  # Append the single row for employees with single entries

# Write the rows of employees with multiple entries at the bottom of the ws starting at the specified row
for employee_id, rows in multiple_entries_rows.items():
    if len(rows) > 1:
        for row_data in rows:
            ws.insert_rows(starting_row_for_multiple_entries, 1)  # Insert a row at the specified row for employees with multiple entries
            for col_index, cell_value in enumerate(row_data):
                ws.cell(row=starting_row_for_multiple_entries, column=col_index + 1, value=cell_value)
            starting_row_for_multiple_entries += 1  # Increment the starting row for the next employee

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")



In [337]:
import pandas as pd
from openpyxl import load_workbook
from collections import defaultdict
from openpyxl.utils import get_column_letter
import re

# Define custom functions for columns 14 and 15 based on columns 2, 3, and 4
def calculate_AvgbillN(row):
    # Net Average Bill
    return row['TotalN'] / row['Visits']

def calculate_AvgbillT(row):
    # Total Average Bill
    return row['TotalT'] / row['Visits']

# Load the existing Excel workbook
wb = load_workbook("StaffPerformanceOverviewtest-06-23 test copy.xlsx")

# Select the ws you want to update (assuming it's the first sheet)
ws = wb.active

# Read the data from the ws into a Pandas DataFrame
data_range = ws['A11:O18']  # Assuming data is in columns A to K starting from row 2
data = [[cell.value for cell in row] for row in data_range]

# Create a DataFrame with all the data
df = pd.DataFrame(data, columns=['EmployeeID', 'Visits', 'New', 'RQs', 'Rat', 'ServiceN', 'ServiceT', 'SeriesN', 
                                  'SeriesT', 'ProductsN', 'ProductsT', 'TotalN', 'TotalT', 'AvgbillN', 'AvgbillT'])

# Get the union of EmployeeIDs
all_employee_ids = set(df['EmployeeID'])

# Create a DataFrame with all EmployeeIDs
df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

# Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

# Fill NaN values with 0
merged_data.fillna(0, inplace=True)

# Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
merged_data['Rat'] = pd.to_numeric(merged_data['Rat'], errors='coerce')
merged_data['AvgbillN'] = pd.to_numeric(merged_data['AvgbillN'], errors='coerce')
merged_data['AvgbillT'] = pd.to_numeric(merged_data['AvgbillT'], errors='coerce')

# Consolidate data based on EmployeeID and calculate the sum for each of the columns 2 to 13
consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
    'Visits': 'sum',
    'New': 'sum',
    'RQs': 'sum',
    'Rat': 'sum',
    'ServiceN': 'sum',
    'ServiceT': 'sum',
    'SeriesN': 'sum',
    'SeriesT': 'sum',
    'ProductsN': 'sum',
    'ProductsT': 'sum',
    'TotalN': 'sum',
    'TotalT': 'sum',
    'AvgbillN': 'sum',
    'AvgbillT': 'sum'})

# Calculate the average for Data4, Data13 columns for each individual employee
consolidated_data['Rat'] = consolidated_data['Rat'] / 2

# Calculate the values for columns 14 and 15 using the custom functions based on other columns
consolidated_data['AvgbillN'] = consolidated_data.apply(calculate_AvgbillN, axis=1)
consolidated_data['AvgbillT'] = consolidated_data.apply(calculate_AvgbillT, axis=1)

# Calculate the column letters for the new columns
conditional_column_letter = get_column_letter(ws.max_column + 1)

# Add new column headers
conditional_column_header = "Conditional Column"
ws[conditional_column_letter + '1'] = conditional_column_header

# Create a dictionary to store the count of employee occurrences
employee_count = {}

# Iterate through the rows (starting from the second row since the first row is headers)
for row in range(2, ws.max_row + 1):
    employee_cell = ws.cell(row=row, column=1)  # Assuming employee names are in column 1 (A)
    employee_name = employee_cell.value

    if not employee_name:
        continue

    # Count the occurrences of each employee name
    if employee_name in employee_count:
        employee_count[employee_name] += 1
    else:
        employee_count[employee_name] = 1

# Populate the new conditional column based on employee count
for row in range(2, ws.max_row + 1):
    employee_name = ws.cell(row=row, column=1).value
    conditional_value = 2 if employee_count.get(employee_name, 0) > 1 else 1
    ws.cell(row=row, column=ws.max_column, value=conditional_value)

# Create a list to store rows to be removed
rows_to_remove = []

# Identify rows with a conditional value of 2 and mark them for removal
for row in range(2, ws.max_row + 1):
    conditional_value = ws.cell(row=row, column=ws.max_column).value
    if conditional_value == 2:
        rows_to_remove.append(row)

# Remove the marked rows in reverse order to avoid shifting issues
for row in reversed(rows_to_remove):
    ws.delete_rows(row)

# Delete the conditional column
ws.delete_cols(ws.max_column)

# Determine the last row of data in the worksheet
last_row = ws.max_row

# Put the consolidated data back into the Excel ws
for row_index, row_data in enumerate(consolidated_data.values):
    for col_index, cell_value in enumerate(row_data):
        cell = ws.cell(row=last_row + 1 + row_index, column=col_index + 1)
        cell.value = cell_value

# Save the updated workbook
wb.save("StaffPerformanceOverviewtest-06-23 test copy.xlsx")


***

## Grabbing Folder with multiple reports and completing a transform and merge with all of them exactly how the code above does

In [19]:
import openpyxl
import os
import re

def read_data(sheet):
    # Find the last row with data in column A
    last_row = sheet.max_row
    for row in range(last_row, 2, -1):
        if sheet.cell(row=row, column=1).value is not None:
            last_row = row
            break

    data_range = f'A3:O{last_row}'
    data_range_values = sheet[data_range]
    data = [[cell.value for cell in row] for row in data_range_values]
    return data


def spo_adjustment(spoCR_path, spoNH_path):

    #Library Importation
    import openpyxl
    from openpyxl import load_workbook
    import pandas as pd
    import numpy as np
    import re

    #Loading Workbooks
    wb1 = load_workbook (spoCR_path)
    wb2 = load_workbook(spoNH_path)
    ws1 = wb1.active
    ws2 = wb2.active

    #Appending Data To One Workbook
    for row in ws2.iter_rows(min_row=2, values_only=True): 
        ws1.append(row)

    def remove_labels(sheet):
        # Create a dictionary to keep track of row indices for each 'New' value
        new_rows = {}
    
        # Iterate through rows in reverse order to safely delete rows
        for row_index in range(sheet.max_row, 1, -1):
            cell_value = sheet.cell(row=row_index, column=3).value
            if cell_value == 'New':
                if cell_value not in new_rows:
                    new_rows[cell_value] = [row_index]
                else:
                    new_rows[cell_value].append(row_index)

        # Iterate through the dictionary and delete the second occurrence of 'New'
        for value, indices in new_rows.items():
            if len(indices) > 1:
                sheet.delete_rows(indices[0], 1)

    # Call the function to remove specific rows from the sheet
    remove_labels(ws1)
    
    date_match = re.search(r'(\d{2}-\d{2}-\d{2})', spoCR_path)
    if date_match:
        date = date_match.group(1)

    firstadjust_filename = f'merged_spo_{date}.xlsx'

    # Save the merged data to a new workbook with the generated filename
    merged_workbook = openpyxl.Workbook()
    merged_sheet = merged_workbook.active

    # Define the folder path where you want to save the merged reports
    merged_SPO_folder = 'Merged SPO reports'

    # Construct the full path for the merged file
    merged_file_path = os.path.join(merged_SPO_folder, firstadjust_filename)

    # Save the merged workbook to the specified absolute path
    wb1.save(merged_file_path)
    #---------------------------------------------------------------------------------------------- The code above works the way I want it for the first report.

    # Load the merged file to complete the rest of the transformations
    second_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    second_adjust_sheet = second_adjust_workbook.active

    # Define the column letter you want to update (e.g., column A)
    column_letter = 'E'

    # Define the value to be replaced and its corresponding replacement
    old = "NA"
    new = "0"

    # Get the column index from the column letter
    column_index = ord(column_letter) - ord('A') + 1

    # Loop through the cells in the selected column and replace the old value with the new value
    for row in second_adjust_sheet.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
        cell = row[0]
        if cell.value == old:
            cell.value = new

    from openpyxl.styles import NamedStyle

    # Create named style for the whole number format
    whole_number_style = NamedStyle(name="whole_number", number_format="0")

    # Columns that need to be formatted as a whole number (example: columns A, B, and D)
    columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

    # Apply the named style to each cell in the selected columns
    for column_letter in columns_to_format:
        for cell in second_adjust_sheet[column_letter]:
            cell.style = whole_number_style

    Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

    columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

    for column_letter in columns_to_format:
        for cell in second_adjust_sheet[column_letter]:
            cell.style = Decimal_Number_Style

    second_adjust_workbook.save(merged_file_path)

    #Second Cell Transformations
    # Load the existing Excel workbook
    third_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    third_adjust_sheet = third_adjust_workbook.active
    # Find the last row in the ws
    last_row = third_adjust_sheet.max_row

    # Create a dictionary to store the rows of employees with multiple entries
    multiple_entries_rows = {}

    # Iterate through the rows and collect the rows of employees with multiple entries
    for ws in [merged_file_path]:
        last_row = third_adjust_sheet.max_row
        for row in range(3, last_row + 1):
            employee_id =  third_adjust_sheet.cell(row=row, column=1).value  # Assuming 'EmployeeID' is in column A
            if employee_id in multiple_entries_rows:
                multiple_entries_rows[employee_id].append([cell.value for cell in third_adjust_sheet[row]])
            else:
                multiple_entries_rows[employee_id] = [[cell.value for cell in third_adjust_sheet[row]]]

    # Clear the existing data in the ws
    third_adjust_sheet.delete_rows(3, last_row)

    # Write the rows of employees with single entries at the top of the ws
    starting_row_for_multiple_entries = 11  # Adjust this variable for different reports
    for employee_id, rows in multiple_entries_rows.items():
        if len(rows) == 1:
            third_adjust_sheet.append(rows[0])  # Append the single row for employees with single entries

    # Write the rows of employees with multiple entries at the bottom of the ws starting at the specified row
    for employee_id, rows in multiple_entries_rows.items():
        if len(rows) > 1:
            for row_data in rows:
                third_adjust_sheet.insert_rows(starting_row_for_multiple_entries, 1)  # Insert a row at the specified row for employees with multiple entries
                for col_index, cell_value in enumerate(row_data):
                    third_adjust_sheet.cell(row=starting_row_for_multiple_entries, column=col_index + 1, value=cell_value)
                starting_row_for_multiple_entries += 1  # Increment the starting row for the next employee

    third_adjust_workbook.save(merged_file_path)

    #Third Cell Transformations
    # Load the merged file to complete the rest of the transformations
    from collections import defaultdict
    from openpyxl.utils import get_column_letter

    # Define custom functions for columns 14 and 15 based on columns 2, 3, and 4
    def calculate_AvgbillN(row):
    # Net Average Bill
        if row['Visits'] != 0:
            return row['TotalN'] / row['Visits']
        else:
            return 0  # Return a default value when 'Visits' is zero


    def calculate_AvgbillT(row):
    # Total Average Bill
        if row['Visits'] != 0:
            return row['TotalT'] / row['Visits']
        else:
            return 0  # Return a default value when 'Visits' is zero

    fourth_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    fourth_adjust_sheet = fourth_adjust_workbook.active

    # Read data from the fourth_adjust_sheet
    data = read_data(fourth_adjust_sheet)  # Automatically reads data starting from A3

    # Create a DataFrame with all the data
    df = pd.DataFrame(data, columns=['EmployeeID', 'Visits', 'New', 'RQs', 'Rat', 'ServiceN', 'ServiceT', 'SeriesN', 
                                    'SeriesT', 'ProductsN', 'ProductsT', 'TotalN', 'TotalT', 'AvgbillN', 'AvgbillT'])

    # Get the union of EmployeeIDs
    all_employee_ids = set(df['EmployeeID'])

    # Create a DataFrame with all EmployeeIDs
    df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

   # Convert the 'EmployeeID' column to string in both dataframes
    df_all_employees['EmployeeID'] = df_all_employees['EmployeeID'].astype(str)
    df['EmployeeID'] = df['EmployeeID'].astype(str)
   
    # Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
    merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

    # Fill NaN values with 0
    merged_data.fillna(0, inplace=True)

    # Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
    merged_data['Rat'] = pd.to_numeric(merged_data['Rat'], errors='coerce')
    merged_data['AvgbillN'] = pd.to_numeric(merged_data['AvgbillN'], errors='coerce')
    merged_data['AvgbillT'] = pd.to_numeric(merged_data['AvgbillT'], errors='coerce')

    # Consolidate data based on EmployeeID and calculate the sum for each of the columns 2 to 13
    consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
        'Visits': 'sum',
        'New': 'sum',
        'RQs': 'sum',
        'Rat': 'sum',
        'ServiceN': 'sum',
        'ServiceT': 'sum',
        'SeriesN': 'sum',
        'SeriesT': 'sum',
        'ProductsN': 'sum',
        'ProductsT': 'sum',
        'TotalN': 'sum',
        'TotalT': 'sum',
        'AvgbillN': 'sum',
        'AvgbillT': 'sum'})

    # Calculate the average for Data4, Data13 columns for each individual employee
    consolidated_data['Rat'] = consolidated_data['Rat'] / 2

    # Calculate the values for columns 14 and 15 using the custom functions based on other columns
    consolidated_data['AvgbillN'] = consolidated_data.apply(calculate_AvgbillN, axis=1)
    consolidated_data['AvgbillT'] = consolidated_data.apply(calculate_AvgbillT, axis=1)

    # Calculate the column letters for the new columns
    conditional_column_letter = get_column_letter(fourth_adjust_sheet.max_column + 1)

    # Add new column headers
    conditional_column_header = "Conditional Column"
    fourth_adjust_sheet[conditional_column_letter + '1'] = conditional_column_header

    # Create a dictionary to store the count of employee occurrences
    employee_count = {}

    # Iterate through the rows (starting from the second row since the first row is headers)
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        employee_cell = fourth_adjust_sheet.cell(row=row, column=1)  # Assuming employee names are in column 1 (A)
        employee_name = employee_cell.value

        if not employee_name:
            continue

        # Count the occurrences of each employee name
        if employee_name in employee_count:
            employee_count[employee_name] += 1
        else:
            employee_count[employee_name] = 1

    # Populate the new conditional column based on employee count
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        employee_name = fourth_adjust_sheet.cell(row=row, column=1).value
        conditional_value = 2 if employee_count.get(employee_name, 0) > 1 else 1
        fourth_adjust_sheet.cell(row=row, column=fourth_adjust_sheet.max_column, value=conditional_value)

    # Create a list to store rows to be removed
    rows_to_remove = []

    # Identify rows with a conditional value of 2 and mark them for removal
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        conditional_value = fourth_adjust_sheet.cell(row=row, column=fourth_adjust_sheet.max_column).value
        if conditional_value == 2:
            rows_to_remove.append(row)

    # Remove the marked rows in reverse order to avoid shifting issues
    for row in reversed(rows_to_remove):
        fourth_adjust_sheet.delete_rows(row)

    # Delete the conditional column
    fourth_adjust_sheet.delete_cols(fourth_adjust_sheet.max_column)

    # Determine the last row of data in the worksheet
    last_row = fourth_adjust_sheet.max_row

    # Put the consolidated data back into the Excel ws
    for row_index, row_data in enumerate(consolidated_data.values):
        for col_index, cell_value in enumerate(row_data):
            cell = fourth_adjust_sheet.cell(row=last_row + 1 + row_index, column=col_index + 1)
            cell.value = cell_value

    # Save the updated workbook
    fourth_adjust_workbook.save(merged_file_path)
    
    # Call spo_adjustment for each batch of reports
if __name__ == '__main__':
# List of report batches (each batch is a list of report paths)
    report_batches = [
    ['Pre reports test/StaffPerformanceOverviewCR06-09-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-09-23.xlsx'],
    ['Pre reports test/StaffPerformanceOverviewCR06-23-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-23-23.xlsx'],
        # Add more batches as needed
    ]
    for batch in report_batches:
        spo_adjustment(batch[0], batch[1])

def process_all_reports():
    reports_folder = 'Pre reports test' 
    report_files = os.listdir(reports_folder)

    for i in range(0, len(report_files), 2):
        spoCR_path = os.path.join(reports_folder, report_files[i])
        spoNH_path = os.path.join(reports_folder, report_files[i + 1])
        spo_adjustment(spoCR_path, spoNH_path)


In [8]:
# Call spo_adjustment for each batch of reports
if __name__ == '__main__':
# List of report batches (each batch is a list of report paths)
    report_batches = [
    ['Pre reports test/StaffPerformanceOverviewCR06-09-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-09-23.xlsx'],
    ['Pre reports test/StaffPerformanceOverviewCR06-23-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-23-23.xlsx'],
        # Add more batches as needed
    ]
    for batch in report_batches:
        spo_adjustment(batch[0], batch[1])

def process_all_reports():
    reports_folder = 'Pre reports test' 
    report_files = os.listdir(reports_folder)

    for i in range(0, len(report_files), 2):
        spoCR_path = os.path.join(reports_folder, report_files[i])
        spoNH_path = os.path.join(reports_folder, report_files[i + 1])
        spo_adjustment(spoCR_path, spoNH_path)


TypeError: '>' not supported between instances of 'list' and 'int'

***

In [ ]:
    # Load the merged file to complete the rest of the transformations
    second_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    second_adjust_sheet = second_adjust_workbook.active

    # Define the column letter you want to update (e.g., column A)
    column_letter = 'E'

    # Define the value to be replaced and its corresponding replacement
    old = "NA"
    new = "0"

    # Get the column index from the column letter
    column_index = ord(column_letter) - ord('A') + 1

    # Loop through the cells in the selected column and replace the old value with the new value
    for row in second_adjust_sheet.iter_rows(min_row=3, min_col=column_index, max_col=column_index):
        cell = row[0]
        if cell.value == old:
            cell.value = new

    from openpyxl.styles import NamedStyle

    # Create named style for the whole number format
    whole_number_style = NamedStyle(name="whole_number", number_format="0")

    # Columns that need to be formatted as a whole number (example: columns A, B, and D)
    columns_to_format = ['B', 'C', 'D']  # Replace with the letters of the columns you want to format

    # Apply the named style to each cell in the selected columns
    for column_letter in columns_to_format:
        for cell in second_adjust_sheet[column_letter]:
            cell.style = whole_number_style

    Decimal_Number_Style = NamedStyle(name="decimal", number_format="#,##0.00")

    columns_to_format = ['E', 'F', 'G', 'H','I','J','K','L','M','N','O']

    for column_letter in columns_to_format:
        for cell in second_adjust_sheet[column_letter]:
            cell.style = Decimal_Number_Style

    second_adjust_workbook.save(merged_file_path)

    #Second Cell Transformations
    # Load the existing Excel workbook
    third_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    third_adjust_sheet = third_adjust_workbook.active
    # Find the last row in the ws
    last_row = third_adjust_sheet.max_row

    # Create a dictionary to store the rows of employees with multiple entries
    multiple_entries_rows = {}

    # Iterate through the rows and collect the rows of employees with multiple entries
    for ws in [merged_file_path]:
        last_row = third_adjust_sheet.max_row
        for row in range(3, last_row + 1):
            employee_id =  third_adjust_sheet.cell(row=row, column=1).value  # Assuming 'EmployeeID' is in column A
            if employee_id in multiple_entries_rows:
                multiple_entries_rows[employee_id].append([cell.value for cell in third_adjust_sheet[row]])
            else:
                multiple_entries_rows[employee_id] = [[cell.value for cell in third_adjust_sheet[row]]]

    # Clear the existing data in the ws
    third_adjust_sheet.delete_rows(3, last_row)

    # Write the rows of employees with single entries at the top of the ws
    starting_row_for_multiple_entries = 11  # Adjust this variable for different reports
    for employee_id, rows in multiple_entries_rows.items():
        if len(rows) == 1:
            third_adjust_sheet.append(rows[0])  # Append the single row for employees with single entries

    # Write the rows of employees with multiple entries at the bottom of the ws starting at the specified row
    for employee_id, rows in multiple_entries_rows.items():
        if len(rows) > 1:
            for row_data in rows:
                third_adjust_sheet.insert_rows(starting_row_for_multiple_entries, 1)  # Insert a row at the specified row for employees with multiple entries
                for col_index, cell_value in enumerate(row_data):
                    third_adjust_sheet.cell(row=starting_row_for_multiple_entries, column=col_index + 1, value=cell_value)
                starting_row_for_multiple_entries += 1  # Increment the starting row for the next employee

    third_adjust_workbook.save(merged_file_path)

    #Third Cell Transformations
    # Load the merged file to complete the rest of the transformations
    from collections import defaultdict
    from openpyxl.utils import get_column_letter


    def calculate_AvgbillN(row):
        TotalN = pd.to_numeric(row['TotalN'], errors='coerce')
    visits = pd.to_numeric(row['Visits'], errors='coerce')
    if visits != 0:
        return TotalN / visits
    else:
        return 0

def calculate_AvgbillT(row):
    total_t = pd.to_numeric(row['TotalT'], errors='coerce')
    visits = pd.to_numeric(row['Visits'], errors='coerce')
    if visits != 0:
        return total_t / visits
    else:
        return 0



    fourth_adjust_workbook = openpyxl.load_workbook(merged_file_path)
    fourth_adjust_sheet = fourth_adjust_workbook.active

    # Read data from the fourth_adjust_sheet
    data = read_data(fourth_adjust_sheet)  # Automatically reads data starting from A3

    # Create a DataFrame with all the data
    df = pd.DataFrame(data, columns=['EmployeeID', 'Visits', 'New', 'RQs', 'Rat', 'ServiceN', 'ServiceT', 'SeriesN', 
                                    'SeriesT', 'ProductsN', 'ProductsT', 'TotalN', 'TotalT', 'AvgbillN', 'AvgbillT'])

    # Get the union of EmployeeIDs
    all_employee_ids = set(df['EmployeeID'])

    # Create a DataFrame with all EmployeeIDs
    df_all_employees = pd.DataFrame({'EmployeeID': list(all_employee_ids)})

   # Convert the 'EmployeeID' column to string in both dataframes
    df_all_employees['EmployeeID'] = df_all_employees['EmployeeID'].astype(str)
    df['EmployeeID'] = df['EmployeeID'].astype(str)
   
    # Merge the original data with the DataFrame containing all EmployeeIDs (outer merge)
    merged_data = pd.merge(df_all_employees, df, on='EmployeeID', how='left')

    # Fill NaN values with 0
    merged_data.fillna(0, inplace=True)

    # Convert 'Data4', 'Data13', and 'Data14' columns to numeric type (ignore errors)
    merged_data['Rat'] = pd.to_numeric(merged_data['Rat'], errors='coerce')
    merged_data['AvgbillN'] = pd.to_numeric(merged_data['AvgbillN'], errors='coerce')
    merged_data['AvgbillT'] = pd.to_numeric(merged_data['AvgbillT'], errors='coerce')

    # Consolidate data based on EmployeeID and calculate the sum for each of the columns 2 to 13
    consolidated_data = merged_data.groupby('EmployeeID', as_index=False).agg({
        'Visits': 'sum',
        'New': 'sum',
        'RQs': 'sum',
        'Rat': 'sum',
        'ServiceN': 'sum',
        'ServiceT': 'sum',
        'SeriesN': 'sum',
        'SeriesT': 'sum',
        'ProductsN': 'sum',
        'ProductsT': 'sum',
        'TotalN': 'sum',
        'TotalT': 'sum',
        'AvgbillN': 'sum',
        'AvgbillT': 'sum'})

    # Calculate the average for Data4, Data13 columns for each individual employee
    consolidated_data['Rat'] = consolidated_data['Rat'] / 2

    # Calculate the values for columns 14 and 15 using the custom functions based on other columns
    consolidated_data['AvgbillN'] = consolidated_data.apply(calculate_AvgbillN, axis=1)
    consolidated_data['AvgbillT'] = consolidated_data.apply(calculate_AvgbillT, axis=1)

    # Calculate the column letters for the new columns
    conditional_column_letter = get_column_letter(fourth_adjust_sheet.max_column + 1)

    # Add new column headers
    conditional_column_header = "Conditional Column"
    fourth_adjust_sheet[conditional_column_letter + '1'] = conditional_column_header

    # Create a dictionary to store the count of employee occurrences
    employee_count = {}

    # Iterate through the rows (starting from the second row since the first row is headers)
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        employee_cell = fourth_adjust_sheet.cell(row=row, column=1)  # Assuming employee names are in column 1 (A)
        employee_name = employee_cell.value

        if not employee_name:
            continue

        # Count the occurrences of each employee name
        if employee_name in employee_count:
            employee_count[employee_name] += 1
        else:
            employee_count[employee_name] = 1

    # Populate the new conditional column based on employee count
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        employee_name = fourth_adjust_sheet.cell(row=row, column=1).value
        conditional_value = 2 if employee_count.get(employee_name, 0) > 1 else 1
        fourth_adjust_sheet.cell(row=row, column=fourth_adjust_sheet.max_column, value=conditional_value)

    # Create a list to store rows to be removed
    rows_to_remove = []

    # Identify rows with a conditional value of 2 and mark them for removal
    for row in range(2, fourth_adjust_sheet.max_row + 1):
        conditional_value = fourth_adjust_sheet.cell(row=row, column=fourth_adjust_sheet.max_column).value
        if conditional_value == 2:
            rows_to_remove.append(row)

    # Remove the marked rows in reverse order to avoid shifting issues
    for row in reversed(rows_to_remove):
        fourth_adjust_sheet.delete_rows(row)

    # Delete the conditional column
    fourth_adjust_sheet.delete_cols(fourth_adjust_sheet.max_column)

    # Determine the last row of data in the worksheet
    last_row = fourth_adjust_sheet.max_row

    # Put the consolidated data back into the Excel ws
    for row_index, row_data in enumerate(consolidated_data.values):
        for col_index, cell_value in enumerate(row_data):
            cell = fourth_adjust_sheet.cell(row=last_row + 1 + row_index, column=col_index + 1)
            cell.value = cell_value

    # Save the updated workbook
    fourth_adjust_workbook.save(merged_file_path)
    
# Call spo_adjustment for each batch of reports
if __name__ == '__main__':
# List of report batches (each batch is a list of report paths)
    report_batches = [
    ['Pre reports test/StaffPerformanceOverviewCR06-09-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-09-23.xlsx'],
    ['Pre reports test/StaffPerformanceOverviewCR06-23-23.xlsx', 'Pre reports test/StaffPerformanceOverviewNH06-23-23.xlsx'],
        # Add more batches as needed
    ]
    for batch in report_batches:
        spo_adjustment(batch[0], batch[1])

def process_all_reports():
    reports_folder = 'Pre reports test' 
    report_files = os.listdir(reports_folder)

    for i in range(0, len(report_files), 2):
        spoCR_path = os.path.join(reports_folder, report_files[i])
        spoNH_path = os.path.join(reports_folder, report_files[i + 1])
        spo_adjustment(spoCR_path, spoNH_path)
